# Section 3.4: Single Model Numerical Diagnostics Activities

In [1]:
import os
import arviz as az

# Change working directory
if os.path.split(os.getcwd())[-1] != "notebooks":
    os.chdir(os.path.join(".."))

NETCDF_DIR = "inference_data"

In [2]:
az.style.use('arviz-white')

## Activity: Calculate $\hat{R}$ and $S_{\text{eff}}$ for our previous analysis
Load each dataset and use ArviZ to calculate the plots for each of the datasets. For Effective Sample Size compare to the count of simulation samples to see how the two compare. Refer back to the traceplots and autocorrelation plots to see how the numerical diagnostics and visual diagnostics correlate.

We've copied one of the examples here for reference.

### Example 1: data_bad_init.nc Numerical Diagnostics 

In [3]:
data_bad_init = az.from_netcdf(os.path.join(NETCDF_DIR, "data_bad_init.nc"))

In [4]:
# Rhat
az.rhat(data_bad_init)
# 3.061 > 1 => variances between the multiple chains don't really match (not converged)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 3.061

In [5]:
# Effective Sample Size
az.effective_sample_size(data_bad_init)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 2.335

In [12]:
# Compare Effective Sample Size to number of samples/draws across all chains
coords = data_bad_init.posterior.coords
print(f"the number of draws: {coords['draw'].shape[0]}")
print(f"the number of chains: {coords['chain'].shape[0]}")
num_computational_draws = coords["draw"].shape[0] * coords["chain"].shape[0]
num_computational_draws 

the number of draws: 200
the number of chains: 2


400

out of 400, only got around 2 => a pretty failed inference run

### Exercise 2: data_good_init.nc Numerical Diagnostics

with a better initialization

In [7]:
# Rhat
data_good_init = az.from_netcdf(os.path.join(NETCDF_DIR, "data_good_init.nc"))
az.rhat(data_good_init)
# Rhat closed to 1. better mixing. verify with the traceplot & auto plot

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 1.235

![data_good_init_traceplot.png](../../img/data_good_init_traceplot.png)

In [8]:
# Effective Sample Size
az.effective_sample_size(data_good_init)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 6.499

In [9]:
# Compare Effective Sample Size to number of samples/draws across all chains
coords = data_good_init.posterior.coords
num_computational_draws = coords["draw"].shape[0] * coords["chain"].shape[0]
num_computational_draws 

400

Out of 400, around 6. Not good enough. Even though the chain seems mixing well, in between the stepsize can be problematic: e.g. too large.

### Exercise 3: data_good_init_long_chain.nc Numerical Diagnostics

In [13]:
# Rhat

data_good_init_long_chain = az.from_netcdf(os.path.join(NETCDF_DIR, "data_good_init_long_chain.nc"))
az.rhat(data_good_init_long_chain)
# R ~ 1. mixed well.

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 1.002

In [15]:
# Effective Sample Size
az.effective_sample_size(data_good_init_long_chain)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 2.251e+03

In [16]:
# Compare Effective Sample Size to number of samples/draws across all chains
coords = data_good_init_long_chain.posterior.coords
num_computational_draws = coords["draw"].shape[0] * coords["chain"].shape[0]
num_computational_draws 

40000

out of 40000, we got around 2250 good samples. Took a lot of time

### Exercise 4: data_tuned_mh.nc Numerical Diagnostics

from PyMC3

In [18]:
data_tuned_mh = az.from_netcdf(os.path.join(NETCDF_DIR, "data_tuned_mh.nc"))
# Rhat
az.rhat(data_tuned_mh)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 1.002

In [19]:
# Effective Sample Size
az.effective_sample_size(data_tuned_mh)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 5.025e+03

In [20]:
coords = data_tuned_mh.posterior.coords
num_computational_draws = coords["draw"].shape[0] * coords["chain"].shape[0]
num_computational_draws 

40000

almost double the number of good samples (around 5000), but still in around 10~20%

### Exercise 5: data_hmc.nc Numerical Diagnostics

A gradient based algorithm. A go-to algorithm these days. NUTS sampler.

In [21]:
data_hmc = az.from_netcdf(os.path.join(NETCDF_DIR, "data_hmc.nc"))
# Rhat
az.rhat(data_hmc)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 1.003

In [22]:
# Effective Sample Size
az.effective_sample_size(data_hmc)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    θ        float64 756.8

In [23]:
coords = data_hmc.posterior.coords
num_computational_draws = coords["draw"].shape[0] * coords["chain"].shape[0]
num_computational_draws 

2000

out of 2000, we got 750 good samples (over 35%)

### Exercise 6: Discussion about numerical diagnostics
Turn to your partner and discuss the results of the numerical diagnostics. 
* Do the numerical diagnostics correlate with your knowledge of MCMC theory?
* Do the diagnostics match your expectation to measuring convergence (mixing) and effective sample size?